<a href="https://colab.research.google.com/github/Otobi1/Predictors-of-Breast-Cancer-Recurrence/blob/master/Predictors_of_Cancer_Recurrence_Production_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the necessary libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import collections
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
import pickle
 
import warnings
warnings.filterwarnings('ignore')

print ('Libraries successfully imported!')

Libraries successfully imported!


In [2]:
# Downloading the data directly from the Repo - http://archive.ics.uci.edu/ml/datasets/Breast+Cancer

!wget http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data

--2021-04-29 17:46:03--  http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18654 (18K) [application/x-httpd-php]
Saving to: ‘breast-cancer.data.2’

breast-cancer.data. 100%[===================>]  18.22K  --.-KB/s    in 0.06s   

2021-04-29 17:46:03 (289 KB/s) - ‘breast-cancer.data.2’ saved [18654/18654]



In [3]:
# Naming the feature columns, originally unnamed
# The meaning of each feature is detailed here on pg 2 https://www.causeweb.org/usproc/sites/default/files/usclap/2018-1/Predictors_for_Breast_Cancer_Recurrence.pdf

df = pd.read_csv('breast-cancer.data', sep=',', names=['RecClass', 'Age', 'Menopause',
                                                       'TumorSize', 'InvNodes', 'NodeCaps',
                                                       'DegMalig', 'Breast', 'Quadrant', 'Radiation']) 

# Shuffle data 

df = df.sample(frac = 1).reset_index(drop = True) 

# Make a copy of the data

data = df.copy()

In [4]:
# Objective 

"""To predict breast cancer recurrence based on selected features"""

'To predict breast cancer recurrence based on selected features'

In [5]:
# Checking the number of observations and features

data.shape

(286, 10)

In [6]:
# Checking the data type, existence of null cases within the observations

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   RecClass   286 non-null    object
 1   Age        286 non-null    object
 2   Menopause  286 non-null    object
 3   TumorSize  286 non-null    object
 4   InvNodes   286 non-null    object
 5   NodeCaps   286 non-null    object
 6   DegMalig   286 non-null    int64 
 7   Breast     286 non-null    object
 8   Quadrant   286 non-null    object
 9   Radiation  286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


In [7]:
# Listing out the feature labels 

data.keys()

Index(['RecClass', 'Age', 'Menopause', 'TumorSize', 'InvNodes', 'NodeCaps',
       'DegMalig', 'Breast', 'Quadrant', 'Radiation'],
      dtype='object')

In [8]:
# Creation of dummy variables for the various features and prevention of the dummy variable trap (multicollinearity)

# Converting the Age group variable into dummy variables and dropping the first column of the AgeGroup category

# Original unique Age group categories '60-69', '40-49', '30-39', '50-59', '70-79', '20-29' (20 - 29 AgeGroup is dropped to prevent Multicollinearity)

Age = pd.get_dummies(data['Age'], drop_first=True)

data = data.drop('Age', axis=1)

Age = Age.add_prefix('AgeGroup ')

data = pd.concat([data, Age], axis=1)

In [9]:
# Converting the Menopause variable into dummy variables and dropping the first column of the Menopause category

# Original unique Menopause categories 'ge40', 'premeno', 'lt40' (ge40 gets dropped)

Menopause = pd.get_dummies(data['Menopause'], drop_first=True)

data = data.drop('Menopause', axis=1)

Menopause = Menopause.add_prefix('Menopause ')

data = pd.concat([data, Menopause], axis=1)

In [10]:
# Converting the TumorSize variables into dummies and droping the first column 

# Original unique TumorSize categories '30-34', '15-19', '25-29', '20-24', '0-4', '35-39', '10-14','40-44', '50-54', '45-49', '5-9' (0-4 dropped)

TumorSize = pd.get_dummies(data['TumorSize'], drop_first=True)

data = data.drop('TumorSize', axis=1)

TumorSize = TumorSize.add_prefix('TumorSize ')

data = pd.concat([data, TumorSize], axis=1)

In [11]:
# Converting the InvNodes variables into dummies and droping the first column 

# Original unique InvNodes categories '0-2', '12-14', '3-5', '15-17', '6-8', '9-11', '24-26' (0-2 dropped)

InvNodes = pd.get_dummies(data['InvNodes'], drop_first=True)

data = data.drop('InvNodes', axis=1)

InvNodes = InvNodes.add_prefix('InvNodes ')

data = pd.concat([data, InvNodes], axis=1)

In [12]:
# Converting the NodeCaps variables into dummies and droping the first column 

# Original unique NodeCaps categories 'no', 'yes', '? (freaking ? dropped, nice)

NodeCaps = pd.get_dummies(data['NodeCaps'], drop_first=True)

data = data.drop('NodeCaps', axis=1)

NodeCaps = NodeCaps.add_prefix('NodeCaps ')

data = pd.concat([data, NodeCaps], axis=1)

In [13]:
# Converting the Breast variables into dummies and droping the first column 

# Original unique Breast categories 'left', 'right', (left dropped)

Breast = pd.get_dummies(data['Breast'], drop_first=True)

data = data.drop('Breast', axis=1)

Breast = Breast.add_prefix('Breast ')

data = pd.concat([data, Breast], axis=1)

In [14]:
# Converting the Quadrant variables into dummies and droping the first column 

# Original unique Quadrant categories 'left_low', 'right_low', 'central', 'left_up', 'right_up', '?' (freaking ? dropped again, nice)

Quadrant = pd.get_dummies(data['Quadrant'], drop_first=True)

data = data.drop('Quadrant', axis=1)

Quadrant = Quadrant.add_prefix('Quadrant ')

data = pd.concat([data, Quadrant], axis=1)

In [15]:
# Converting the Radiation variables into dummies and droping the first column 

# Original unique Radiation categories 'no', 'yes' (no dropped)

Radiation = pd.get_dummies(data['Radiation'], drop_first=True)

data = data.drop('Radiation', axis=1)

Radiation = Radiation.add_prefix('Radiation ')

data = pd.concat([data, Radiation], axis=1)

In [16]:
# Listing out the feature labels 

data.keys()

Index(['RecClass', 'DegMalig', 'AgeGroup 30-39', 'AgeGroup 40-49',
       'AgeGroup 50-59', 'AgeGroup 60-69', 'AgeGroup 70-79', 'Menopause lt40',
       'Menopause premeno', 'TumorSize 10-14', 'TumorSize 15-19',
       'TumorSize 20-24', 'TumorSize 25-29', 'TumorSize 30-34',
       'TumorSize 35-39', 'TumorSize 40-44', 'TumorSize 45-49',
       'TumorSize 5-9', 'TumorSize 50-54', 'InvNodes 12-14', 'InvNodes 15-17',
       'InvNodes 24-26', 'InvNodes 3-5', 'InvNodes 6-8', 'InvNodes 9-11',
       'NodeCaps no', 'NodeCaps yes', 'Breast right', 'Quadrant central',
       'Quadrant left_low', 'Quadrant left_up', 'Quadrant right_low',
       'Quadrant right_up', 'Radiation yes'],
      dtype='object')

In [17]:
# Creating a list of least impact features based on the initial notebook 

features_to_drop = [
                    'AgeGroup 40-49', 'Quadrant left_up',  'Quadrant left_low', 'Quadrant central', 
                    'NodeCaps no', 'TumorSize 15-19', 'AgeGroup 60-69', 'TumorSize 45-49', 'TumorSize 35-39', 
                    'Quadrant right_low', 'TumorSize 40-44', 'AgeGroup 50-59', 'Breast right', 'TumorSize 5-9', 
                    'TumorSize 10-14'
]

In [18]:
# Dropping the least impact columns 

data.drop(columns=features_to_drop, inplace=True) 
data.shape

(286, 19)

In [19]:
data.head()

,RecClass,DegMalig,AgeGroup 30-39,AgeGroup 70-79,Menopause lt40,Menopause premeno,TumorSize 20-24,TumorSize 25-29,TumorSize 30-34,TumorSize 50-54,InvNodes 12-14,InvNodes 15-17,InvNodes 24-26,InvNodes 3-5,InvNodes 6-8,InvNodes 9-11,NodeCaps yes,Quadrant right_up,Radiation yes
0,recurrence-events,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
1,no-recurrence-events,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,recurrence-events,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,no-recurrence-events,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,no-recurrence-events,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# Mapping the input features into the X dataframe and the labels into y

X = data._get_numeric_data()
y = data['RecClass']

# Creating the train and test data splits 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 1919)

In [21]:
# Checking the size and dimensions of the data splits 

print (f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print (f'X_test: {X_test.shape}, y_test: {y_test.shape}')

X_train: (214, 18), y_train: (214,)
X_test: (72, 18), y_test: (72,)


In [22]:
# Exploring the overall class distribution
# To see if the observations and the categories are fairly evenly distributed between the data splits

class_count = dict(collections.Counter(y))
train_class_count = dict(collections.Counter(y_train))
test_class_count = dict(collections.Counter(y_test))

print (f'classes: {class_count}')
print (f"no-rec:rec = {class_count['no-recurrence-events']/class_count['recurrence-events']:.2f}")
print (f'train classes: {train_class_count}')
print (f"train no-rec:rec = {train_class_count['no-recurrence-events']/train_class_count['recurrence-events']:.2f}")
print (f'test classes: {test_class_count}')
print (f"test no-rec:rec = {test_class_count['no-recurrence-events']/test_class_count['recurrence-events']:.2f}")

classes: {'recurrence-events': 85, 'no-recurrence-events': 201}
no-rec:rec = 2.36
train classes: {'no-recurrence-events': 150, 'recurrence-events': 64}
train no-rec:rec = 2.34
test classes: {'no-recurrence-events': 51, 'recurrence-events': 21}
test no-rec:rec = 2.43


In [23]:
# Standardising the data (mean = 0, std = 1) using the training data

X_scaler = StandardScaler().fit(X_train)

# Not necessary to standardise the data using the StandardScaler class of Sklearn as the orders of magnitude are in single digits

In [24]:
# Applying the scaler on training and test data (not necessary to standardise outputs for classification)

X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [25]:
# Check (mean should approx 0 and std should be approx 1)

print (f'X_train[0]: mean: {np.mean(X_train[:, 0], axis = 0):.1f}, std: {np.std(X_train[:, 0], axis=0):.1f}')
print (f'X_test[1]: mean: {np.mean(X_test[:, 1], axis = 0):.1f}, std: {np.std(X_test[:, 1], axis=0):.1f}')

X_train[0]: mean: 0.0, std: 1.0
X_test[1]: mean: 0.1, std: 1.1


In [26]:
# Base Model 

y.value_counts(normalize=True)

# A base model that predicts no recurrence events would be correct 70% of the time

no-recurrence-events    0.702797
recurrence-events       0.297203
Name: RecClass, dtype: float64

In [27]:
# Logistic Regression 

# Running a pipeline of logistic regression 
pipe = Pipeline(steps=[('lr', LogisticRegression())]) 

# setting parameters
params = {'lr__penalty': ['l1'], 
          'lr__C': [1], 
          'lr__solver': ['liblinear']} 

gs_lr = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy', n_jobs=-2)

# Fitting the logistic regression model on the training data split
gs_lr.fit(X_train, y_train)
gs_lr.best_estimator_

Pipeline(memory=None,
         steps=[('lr',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l1', random_state=None,
                                    solver='liblinear', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [28]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_lr.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_lr.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_lr.score(X_test, y_test)}")

# Inference: this model performs worse than the base model on the test data split

training score: 0.7616822429906542
cross validation score: 0.7062310949788264
test score: 0.6944444444444444


In [29]:
# K-Nearest Neighbors Classifier

# Running a pipeline of KNN classifiers
pipe = Pipeline(steps=[('sc', StandardScaler()), ('knn', KNeighborsClassifier())])

# Setting the parameters
params = {'knn__n_neighbors': [21], 'knn__p': [1]}

gs_knn = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy')

# Fitting the KNN model on the training data split
gs_knn.fit(X_train, y_train)
gs_knn.best_estimator_

Pipeline(memory=None,
         steps=[('sc',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=21, p=1,
                                      weights='uniform'))],
         verbose=False)

In [30]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_knn.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_knn.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_knn.score(X_test, y_test)}")

# Inference: this model performs better than the base model and the logistic regression model

training score: 0.705607476635514
cross validation score: 0.7133696309739868
test score: 0.7083333333333334


In [31]:
# Decision Tree Classifer

# Running a pipeline of Decision Tree Classifiers
pipe = Pipeline(steps=[('tree', DecisionTreeClassifier())])

# Setting the parameters
params = {'tree__max_depth': [6, 8]}

gs_tree = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy')

# Fitting the Decision Tree Model on the training data split
gs_tree.fit(X_train, y_train)
gs_tree.best_estimator_

Pipeline(memory=None,
         steps=[('tree',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=8,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='best'))],
         verbose=False)

In [32]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_tree.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_tree.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_tree.score(X_test, y_test)}")

# Inference: this model likely overfits on the training data split but performs worse than the base model, the logistic regression and the KNN models on the test data split

training score: 0.9018691588785047
cross validation score: 0.7273442226255293
test score: 0.7222222222222222


In [33]:
# Bagging Classifier Model 

# Running a pipeline of Bagging Classifiers
pipe = Pipeline(steps = [('bag', BaggingClassifier())])

# Setting the parameters 
params = {'bag__n_estimators': [200]}

gs_bag = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy')

# Fitting the Bagging Classifier model on the training data split
gs_bag.fit(X_train, y_train)
gs_bag.best_estimator_

Pipeline(memory=None,
         steps=[('bag',
                 BaggingClassifier(base_estimator=None, bootstrap=True,
                                   bootstrap_features=False, max_features=1.0,
                                   max_samples=1.0, n_estimators=200,
                                   n_jobs=None, oob_score=False,
                                   random_state=None, verbose=0,
                                   warm_start=False))],
         verbose=False)

In [34]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_bag.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_bag.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_bag.score(X_test, y_test)}")

# Inference: model overfits on the training data split but performs better than the  than the base model, decision tree and logistic regression

training score: 0.9158878504672897
cross validation score: 0.7272837265577736
test score: 0.75


In [35]:
# Random Forest Model 

# Running a pipeline of Random Forest Classifiers
pipe = Pipeline(steps=[('forest', RandomForestClassifier())])

# Setting the parameters 
params = {'forest__n_estimators': [150], 'forest__max_depth':[15]}

gs_forest = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy')

# Fitting the Random Forest Classifier to the training data split
gs_forest.fit(X_train, y_train)
gs_forest.best_estimator_

Pipeline(memory=None,
         steps=[('forest',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=15, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=150, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [36]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_forest.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_forest.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_forest.score(X_test, y_test)}")

# Inference: model overfits the training data splits but performs better than the base model and logistic regression but comparably to the Bagging Classifier and the KNN

training score: 0.9158878504672897
cross validation score: 0.7273442226255293
test score: 0.7638888888888888


In [37]:
# Support Vector Classififer 

# - Setting up the model 
pipe = Pipeline(steps = [("svc", SVC())])

# - Setting the model parameters 
params = {"svc__C": [3]}

gs_svc = GridSearchCV(pipe, param_grid = params, cv = 5, scoring = "accuracy")

# Fitting the SVC model to the training dataset
gs_svc.fit(X_train, y_train)
gs_svc.best_estimator_

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=3, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [38]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_svc.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_svc.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_svc.score(X_test, y_test)}")

# Inference: model performs quite well on the trainin data split and best performing of the models used on the test dataset

training score: 0.8551401869158879
cross validation score: 0.7343012704174229
test score: 0.7638888888888888


In [39]:
# Voting Classifier Model 

knn_pipe = Pipeline([('ss', StandardScaler()), ('knn', KNeighborsClassifier())])

# Running the pipeline for the Voting Classifier
vote = VotingClassifier([('rand', RandomForestClassifier()), 
                         ('grad', GradientBoostingClassifier()), 
                         ('lr', LogisticRegression()), 
                         ('tree', DecisionTreeClassifier()), 
                         ('bag', BaggingClassifier()), 
                         ('ada', AdaBoostClassifier()), 
                         ('extra', ExtraTreesClassifier()), 
                         ('knn_pipe', knn_pipe)], voting = 'soft')

# Setting the parameters 
vote_params = {'rand__n_estimators': [150], 
               'rand__max_depth': [15], 
               'grad__n_estimators': [300], 
               'tree__max_depth': [8], 
               'bag__n_estimators': [200], 
               'ada__n_estimators': [10], 
               'extra__n_estimators': [600], 
               'knn_pipe__knn__n_neighbors': [21], 
               'lr__penalty': ['l1'], 
               'lr__C': [1], 
               'lr__solver': ['liblinear']}

gs_vc = GridSearchCV(vote, param_grid=vote_params, cv=5, scoring='accuracy')

# Fitting the voting classifier to the training data split
gs_vc.fit(X_train, y_train)
gs_vc.best_estimator_

VotingClassifier(estimators=[('rand',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=15,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
        

In [40]:
# Scoring the model (training score, cross validation and test score)

print (f'training score: {gs_vc.score(X_train, y_train)}')
print (f"cross validation score: {cross_val_score(gs_vc.best_estimator_, X, y, cv=5).mean()}")
print (f"test score: {gs_vc.score(X_test, y_test)}")

# Inference, voting classifier performs slightly worse than the SVC on the test data and almost overfits the training data. 
# Also as voting classifiers are ensemble, they are difficult to interprete

training score: 0.9112149532710281
cross validation score: 0.7308529945553539
test score: 0.75


In [41]:
# Pickling the best model 

with open('prod_model.pkl', 'wb') as f_out:
  pickle.dump(gs_forest, f_out)
  f_out.close()